In [1]:
import orjson
import numpy as np
import math

In [21]:
with open("../resources/data_final.json", "rb") as f:
    dataset = orjson.loads(f.read())

In [22]:
np.random.seed(0)
id_train = np.random.choice(len(dataset), math.ceil(len(dataset)/2))

In [23]:
id_train.shape

(48,)

In [24]:
id_test = np.asarray(list(set(range(len(dataset))).difference(set(id_train))))

In [25]:
id_test.shape

(60,)

In [8]:
# https://github.com/ThilinaRajapakse/simpletransformers#question-answering

In [41]:
from simpletransformers.question_answering import QuestionAnsweringModel
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Create dummy data to use for training.
train_data = [dataset[id] for id in id_train]

# Create the QuestionAnsweringModel
model = QuestionAnsweringModel('bert', 'mrm8488/bert-base-portuguese-cased-finetuned-squad-v1-pt', args={'reprocess_input_data': True, 'overwrite_output_dir': True})
model.lazy_loading = True

# The list can also be used directly
model.train_model(train_data)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
convert squad examples to features:   0%|          | 0/48 [00:00<?, ?it/s]/home/giuseppe/PycharmProjects/ple_chat_bot/venv/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1324: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
add example index and unique id: 100%|██████████| 48/48 [00:00<00:00, 90200.09it/s]
/home/giuseppe/PycharmProjects/ple_chat_bot/venv/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.ht

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 3.95 GiB total capacity; 2.90 GiB already allocated; 27.38 MiB free; 3.08 GiB reserved in total by PyTorch)

In [27]:
# Evaluate the model. (Being lazy and evaluating on the train data itself)
result, text = model.eval_model(train_data)

print(result)
print(text)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
convert squad examples to features:   0%|          | 0/48 [00:00<?, ?it/s]/home/giuseppe/PycharmProjects/ple_chat_bot/venv/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1324: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
add example index and unique id: 100%|██████████| 48/48 [00:00<00:00, 189039.05it/s]



{'correct': 0, 'similar': 35, 'incorrect': 0, 'eval_loss': -12.676215277777779}
{'correct_text': {}, 'similar_text': {'1243188607': {'truth': 'A equiparação funcionará de acordo com as regras constantes no PPC de cada curso e em acordo com a resolução 425/2010\xa0 CEPE/UFRPE. A equiparação de estágio pode ser feita, se prevista no PPC do curso. Caso não seja prevista, o CCD e NDE do curso podem de forma ágil possibilitar a equiparação, já para o PLE.', 'predicted': '', 'question': 'Como funciona a equiparação? Todos os cursos, incluindo licenciaturas, podem ter?'}, '1243188610': {'truth': 'A equiparação é um mecanismo legal. Porém, segundo a lei de estágio ela deve constar no PPC do curso. No PLE os cursos farão um projeto para anexar ao PPC. Procure sua coordenação e fale sobre seu caso específico. Se a atividade for compatível com o ESO, a coordenação poderá aceitar a equiparação.', 'predicted': '', 'question': 'Com relação à\xa0equiparação do estágio obrigatório, é possível equipar

In [39]:
# Making predictions using the model.
to_predict = np.asarray([dataset[id_test[0]]])
predict = model.predict(to_predict)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
convert squad examples to features:   0%|          | 0/1 [00:00<?, ?it/s]/home/giuseppe/PycharmProjects/ple_chat_bot/venv/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1324: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 856.33it/s]


In [40]:
for pred in predict:
    print(pred)

[{'id': '1243188563', 'answer': ['', 'estudantes novatos', 'matriculas discentes nauaeadtecsecao idos alunos ingressantesart unidade academica educacao distancia tecnologiauaeadtec matriculas estudantes novatos', 'estudantes novatos referentes editais', 'selecao', 'novatos', 'discentes nauaeadtecsecao idos alunos ingressantesart unidade academica educacao distancia tecnologiauaeadtec matriculas estudantes novatos', 'alunos ingressantesart unidade academica educacao distancia tecnologiauaeadtec matriculas estudantes novatos', 'matriculas discentes nauaeadtecsecao idos alunos ingressantesart unidade academica educacao distancia tecnologiauaeadtec matriculas estudantes novatos referentes editais', 'estudantes novatos referentes editais selecao', 'capitulo ii procedimentos matriculas discentes nauaeadtecsecao idos alunos ingressantesart unidade academica educacao distancia tecnologiauaeadtec matriculas estudantes novatos', 'novatos referentes editais', 'discentes nauaeadtecsecao idos aluno